>**Generate a word frequency list**
>
>This notebook loads the vocabulary learned from the MIMIC-III free-text notes and uses it as a starting point to generate a custom word frequency list. We expand the vocabulary by adding names of common drugs (including generic, brand, and slang names) and local mental health organisations.
The word frequency list is generated by parsing the whole dataset and appending to an empty list every word that is known to the vocabulary.

In [1]:
import pandas as pd
import json

# Project imports
from self_harm_triage_notes.config import *
from self_harm_triage_notes.text_utils import *

In [2]:
dev_data_filename = "rmh_2012_2017_dev" # rmh_2012_2017_dev, lvrh_2012_2017_dev

___
# Prepare data
### Load RMH development data

In [3]:
df = pd.read_parquet(interim_data_dir / (dev_data_filename + ".parquet"), engine="pyarrow")
print(df.shape)
df.head()

(319288, 15)


,uid,sex,age,arrival_method,arrival_date,year,triage_note,SH,SI,AOD_OD,audit_case,source_system,quarter,length,val_fold
0,RMH-1,female,64.0,other,2012-01-08 00:35:00,2012,"SOB for 5/7, been to GP given prednisolone, co...",Negative,Negative,Negative,NaN,Symphony,2012Q1,140,4
1,RMH-2,male,31.0,other,2012-01-08 00:41:00,2012,"pt has lac down right forehead, to eyebrow, wi...",Negative,Negative,Negative,NaN,Symphony,2012Q1,107,1
2,RMH-3,male,19.0,road ambulance,2012-01-08 00:52:00,2012,"pt expect MBA, trapped for 45mins, #right femu...",Negative,Negative,Negative,NaN,Symphony,2012Q1,74,1
3,RMH-5,female,25.0,other,2012-01-08 01:23:00,2012,generalised abdo pain and associated headache ...,Negative,Negative,Negative,NaN,Symphony,2012Q1,196,5
4,RMH-6,female,18.0,other,2012-01-08 01:37:00,2012,abdo pain associated with constipation. Pt se...,Negative,Negative,Negative,NaN,Symphony,2012Q1,134,5


In [4]:
print_token_counts(count_tokens(df.triage_note))
print_token_counts(count_tokens(df.triage_note, valid=True))

The corpus contains 227911 unique tokens (6594864 tokens in total).
The corpus contains 212259 unique tokens (6153769 tokens in total).


### Pre-process

In [5]:
# Pre-processing
df['preprocessed_triage_note'] = df.triage_note.apply(preprocess)

print_token_counts(count_tokens(df.preprocessed_triage_note))
print_token_counts(count_tokens(df.preprocessed_triage_note, valid=True))

The corpus contains 144936 unique tokens (6761504 tokens in total).
The corpus contains 134722 unique tokens (6238239 tokens in total).


### Tokenise

In [6]:
# Create tokenised text
df['tokenized_triage_note'] = tokenize_step1(df.preprocessed_triage_note)

print_token_counts(count_tokens(df.tokenized_triage_note))
print_token_counts(count_tokens(df.tokenized_triage_note, valid=True))
df.to_parquet(interim_data_dir / (dev_data_filename + "_tokenised_step1.parquet"), engine="pyarrow")

/opt/homebrew/Caskroom/miniforge/base/envs/self-harm-triage-notes/lib/python3.11/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
/opt/homebrew/Caskroom/miniforge/base/envs/self-harm-triage-notes/lib/python3.11/site-packages/spacy/util.py:1198: FutureWarning: Possible set union at position 6338
  return re.compile(expression)


The corpus contains 102953 unique tokens (7630368 tokens in total).
The corpus contains 95553 unique tokens (6263764 tokens in total).


In [7]:
# df = pd.read_parquet(data_interim_dir / (dev_data_filename + "_tokenised_step1.parquet"), engine="pyarrow")
# print_token_counts(count_tokens(df.tokenized_triage_note))
# print_token_counts(count_tokens(df.tokenized_triage_note, valid=True))

___
# Baseline vocabulary

### General English

In [8]:
# List all available files
files_eng = [str(path) for path in scowl_dir.glob("english-words*")]
files_aus = [str(path) for path in scowl_dir.glob("australian-words*")]
files_upp = [str(path) for path in scowl_dir.glob("english-upper*")]
files = files_eng + files_aus + files_upp

# Select word lists <= 70
files = [f for f in files if int(f.split('.')[-1]) <= 70]

# Create an empty list
english_words = []

# Load and append lists of words to the common list
for filename in files:
    with open (filename, 'rb') as f:
        words = f.read().decode(errors='ignore')

    english_words.extend(words.replace('\n', ' ').lower().split())

    print("Total number of words: %d" % len(english_words))

# Convert to a set
english_words = set(english_words)
print("English, australian, and english upper word lists (<= 70) result in a total of %d words." % 
      len(english_words))

Total number of words: 7951
Total number of words: 12324
Total number of words: 36120
Total number of words: 72223
Total number of words: 85661
Total number of words: 118931
Total number of words: 125322
Total number of words: 131555
Total number of words: 131779
Total number of words: 133092
Total number of words: 133314
Total number of words: 133360
Total number of words: 133556
Total number of words: 134104
Total number of words: 134906
Total number of words: 136391
Total number of words: 136877
Total number of words: 143176
Total number of words: 143915
Total number of words: 144133
Total number of words: 154476
Total number of words: 154489
English, australian, and english upper word lists (<= 70) result in a total of 152186 words.


### AMT Australian ED reference set

In [9]:
# Load reference set
df_amt = pd.read_csv(amt_ed_path, sep='\t')
df_amt.columns = df_amt.columns.str.lower().str.replace(' ', '_')

fully_spec = {
    word
    for line in df_amt.fully_specified_name.dropna().str.lower().str.replace('(disorder)', '').str.replace('(finding)', '').apply(lambda x: re.sub("[:,()/]", " ", x).strip()).tolist()
    for word in line.split()
}
print(len(fully_spec), "fully specified terms")

# pref_terms = {
#     word
#     for line in df_amt.preferred_term.dropna().str.lower().apply(lambda x: re.sub("[:,()/]", " ", x).strip()).tolist()
#     for word in line.split()
# }
# print(len(pref_terms), "preferred terms")

# acc_syn = [
#     word
#     for line in df_amt.acceptable_synonyms.dropna().str.lower().str.replace(' - ', '').apply(lambda x: re.sub("[:,()/\\\]", " ", x).strip()).tolist()
#     for word in line.split()
# ]

ed_terms = fully_spec#.union(pref_terms)#set(fully_spec + pref_terms)# + acc_syn)
print("Aus ED ref set contains %d words." % len(ed_terms))

23311 fully specified terms
Aus ED ref set contains 23311 words.


### AMT Medicinal and trade product reference sets

In [10]:
# Medicinal products
df_amt = pd.read_csv(amt_mp_path, sep='\t')
df_amt.columns = df_amt.columns.str.lower().str.replace(' ', '_')

medicinal_products = [
    word
    for line in df_amt.preferred_term.dropna().str.lower().apply(lambda x: re.sub("[+,()]", "", x).strip()).tolist()
    for word in line.split()
    if word.isalpha()
]

# Trade products
df_amt = pd.read_csv(amt_tp_path, sep='\t')
df_amt.columns = df_amt.columns.str.lower().str.replace(' ', '_')

trade_products = [
    word
    for line in df_amt.preferred_term.dropna().str.lower().apply(lambda x: re.sub("[/,()]", " ", x).strip()).tolist()
    for word in line.split()
    if word.isalpha()
]

drug_names = set(medicinal_products + trade_products)
print("AMT medicinal and trade product ref sets contains %d words." % len(drug_names))

AMT medicinal and trade product ref sets contains 7858 words.


### Names of places in Vic

In [11]:
# Load the list of places in Victoria
with open (vic_path, 'r') as f:
    vic_places = f.read()
    
vic_places = set(vic_places.replace('\n', ' ').lower().split())
print("List of names of places in Vic contains %d words." % len(vic_places))

List of names of places in Vic contains 1510 words.


### Local medical organisations

In [12]:
# List of local medical organisations
med_orgs = [
    "ecatt", 
    "orygen", 
    "saapu", 
    "vccc", # Victorian Comprehensive Cancer Centre
    "pmcc", # Peter MacCallum Cancer Centre
    "metcall", 
    "wgh", # West Gippsland Hospital
    "rmh", # Royal Melbourne Hospital
    "rwh", # Royal Women's Hospital
    "rdns", # Royal District Nursing Service
    "emh",
    "alfred",
    "epworth",
    "jfh",
 ]

### Other terms

In [13]:
# List of units
units = [
    "km",
    "kms",
    "mins",
    "min",
    "kph",
    "kmh",
    "sec",
    "yr",
    "yrs",
    "mph",
    "mls",
    "mgs",
    "l",
]

# List of highly specific medical terms
med_terms = [
    "headstrike",
    "painfree",
    "section351",
    "penthrane",
    "normotensive",
    "ictal",
    "cspine",
    "laminectomy",
    "holter",
    "unwellness",
    "presyncopal",
    "hemi",
]   
#     "unrousable",
#     "unrecordable",
#     "batcall",
#     "acopia", 
#     "daswest",
#     "neurovasc", 
#     "vasc", 
#     "bibp", 
#     "headstrike", 
#     "n&v", 
#     "v&d",
#     "d&v",
#     "n&v&d",
#     "foosh", 
#     "esrf", 
#     "urti", 
#     "lrti",
#     "cwms", 
#     "lvf", 
#     "stml", 
#     "permacath", 
#     "section351", 
#     "weightbear", 
#     "creps", 
#     "warfarinised", 
#     "midzone", 
#     "burnshield", 
#     "aperient", 
#     "haemoserous"
# ]

# Common incorrectly spelled words contained in med7
incorrect_spelling = [
    "diarrhea",
]
#     'sucidal', 
#     'sucide', 
#     'intermitantly',
#     'intermitent',
#     'intermitently',
#     'intermittant',
#     'intermittantly',
#     'intermittenly', 
#     'spontaenous',
#     'spontan',
#     'spontanious',
#     'spontaniously',
#     'spontanous',
#     'spontanously'

> `soboe` = `sobe` = shortness of breath on exertion
>
> `n&v` = nausea & vomiting
>
> `foosh` = falling on outstretched hand
>
> `cabg` = `cabgs` = coronary artery bypass surgery 
>
> `exac` = exacerbation
>
> `oab` = `oabs` = `abs` = [oral] antibiotics
>
> `hlcnh` = high-level care nursing home
>
> `nv` = `nvasc` = `n'vasc` = `neurovasc` = neuro-vascular
>
> `esrf` = end-stage renal failure 
>
> `esr` = erythrocyte Sedimentation Rate
> 
> `urti` = upper respiratory tract infection 
>
> `lrti` = lower respiratory tract infection
>
> `uti` = urinary tract infection
>
> `fpx4` = `fp4` = focus point at 4mm ?? 
>
> `cwms` = `cms` = circulation [warmth] motion sensation

abbs = {
    'sob': 'shortness of breath',
    'loc': 'loss of consciousness',
    'lac': 'laceration',
    'gtn': 'glyceryl trinitrate',
    'ami': 'acute myocardial infarction',
    'mva': 'motor vehicle accident',
    'nad': 'no abnormality detected',
    'mas': 'motor assessment scale',
    'cva': 'cerebrovascular accident'
}

In [14]:
# Create a baseline vocabulary
base_vocab = english_words

# Add ED-specific terms
base_vocab.update(ed_terms)

# Add drug names
base_vocab.update(drug_names)

# Add places in Vic
base_vocab.update(vic_places)

# Add local medical organisations
base_vocab.update(med_orgs)

# Add highly specific medical terms
base_vocab.update(med_terms)

# Add units
base_vocab.update(units)

# Remove incorrectly spelt words
base_vocab = {v for v in base_vocab if v not in incorrect_spelling}

print("Combined base vocabulary contains %d  words." % len(base_vocab))

Combined base vocabulary contains 172551  words.


### Abbreviations

In [15]:
df_abbs = pd.read_excel(abbr_path, header=0)
df_abbs.columns = ['abbreviation_raw', 'meaning']
df_abbs['abbreviation_lower'] = df_abbs.abbreviation_raw.str.lower()

print("List of abbreviations contains %d words." % df_abbs.shape[0])

List of abbreviations contains 2272 words.


___
# Filter vocabulary and abbreviations
### Filter the base vocab

In [16]:
# Create a vocabulary of known tokens
counts = count_tokens(df.tokenized_triage_note, valid=True)
filtered_vocab = set(k for k in counts.keys() if k in base_vocab)
print(len(filtered_vocab))

19170


### Filter abbreviations

In [17]:
# Filter out abbreviations that do not occur in the data in their raw form
counts = count_tokens(df.triage_note, valid=True)
filtered_abbs = set(df_abbs.loc[df_abbs.abbreviation_raw.isin(counts), 'abbreviation_lower'])
print(len(filtered_abbs))

999


### Re-tokenise

In [18]:
# Re-tokenise text
df.tokenized_triage_note = tokenize_step2(df.tokenized_triage_note, filtered_vocab.union(filtered_abbs))

# Update the dictionary of tokens with counts
counts = count_tokens(df.tokenized_triage_note, valid=True)
word_list = Counter({k:v for k,v in counts.items() if k in base_vocab or k in filtered_abbs})
len(word_list)

20043

In [19]:
# Most common words
sorted(word_list.items(),  key=lambda item: item[1], reverse=True)

[('pain', 176342),
 ('to', 154530),
 ('nil', 124922),
 ('and', 124604),
 ('with', 104258),
 ('on', 77353),
 ('phx', 75126),
 ('at', 64495),
 ('of', 62585),
 ('left', 58700),
 ('for', 57865),
 ('hx', 57663),
 ('in', 54578),
 ('pt', 52997),
 ('right', 50854),
 ('from', 41879),
 ('post', 39195),
 ('chest', 35463),
 ('ago', 32997),
 ('gcs', 32718),
 ('by', 30957),
 ('pmhx', 29858),
 ('gp', 29404),
 ('sob', 28936),
 ('abdo', 28725),
 ('denies', 27377),
 ('has', 26849),
 ('rate', 26849),
 ('l', 26619),
 ('today', 26514),
 ('not', 26272),
 ('nausea', 25902),
 ('states', 25478),
 ('back', 25117),
 ('last', 24907),
 ('loc', 24743),
 ('o/a', 24472),
 ('fall', 23902),
 ('heart', 23654),
 ('no', 23551),
 ('since', 22940),
 ('r', 22739),
 ('now', 21889),
 ('pmh', 21599),
 ('c/o', 21565),
 ('triage', 21402),
 ('this', 20711),
 ('onset', 19962),
 ('av', 19604),
 ('ht', 17962),
 ('swelling', 17730),
 ('worse', 17019),
 ('had', 17017),
 ('blood', 16934),
 ('hours', 16782),
 ('x', 16525),
 ('headache', 

In [20]:
# Most common words with 3 letters
sorted({k: v for k,v in word_list.items() if len(k)==3}.items(),  
       key=lambda item: item[1], reverse=True)

[('nil', 124922),
 ('and', 124604),
 ('phx', 75126),
 ('for', 57865),
 ('ago', 32997),
 ('gcs', 32718),
 ('sob', 28936),
 ('has', 26849),
 ('not', 26272),
 ('loc', 24743),
 ('o/a', 24472),
 ('now', 21889),
 ('pmh', 21599),
 ('c/o', 21565),
 ('had', 17017),
 ('leg', 14597),
 ('arm', 14422),
 ('the', 12430),
 ('lac', 11439),
 ('but', 11168),
 ('htn', 10242),
 ('eye', 8627),
 ('car', 8008),
 ('non', 7702),
 ('pmx', 7581),
 ('gtn', 7408),
 ('was', 6963),
 ('d/c', 6577),
 ('ami', 6426),
 ('due', 6206),
 ('lmo', 6074),
 ('per', 5794),
 ('hit', 5772),
 ('hip', 5694),
 ('rmh', 5409),
 ('mva', 5259),
 ('nad', 5195),
 ('out', 5086),
 ('off', 4814),
 ('ccf', 4421),
 ('mas', 4186),
 ('cva', 4144),
 ('few', 4100),
 ('red', 4023),
 ('bsl', 4010),
 ('ppm', 3891),
 ('uti', 3671),
 ('ihd', 3658),
 ('all', 3647),
 ('low', 3634),
 ('rom', 3618),
 ('see', 3523),
 ('ear', 3431),
 ('hot', 3402),
 ('use', 3336),
 ('jaw', 3273),
 ('ecg', 3194),
 ('any', 3160),
 ('ccp', 3111),
 ('dry', 3103),
 ('bed', 2921),
 

### Save the word list and vocabulary

In [21]:
with open(spell_corr_dir / (dev_data_filename + "_amt6_word_freq_list.json"), 'w') as f:
    json.dump(word_list, f)
with open(spell_corr_dir / (dev_data_filename + "_amt6_vocab.json"), 'w') as f:
    json.dump(list(word_list.keys()), f)

### Save dataset

In [22]:
df.to_parquet(interim_data_dir / (dev_data_filename + "_amt6_nospellcorr.parquet"), engine="pyarrow")